# List corrupted images

In [ ]:
import os

# Define the root directory
root_dir = "Streetview_data"

# List to store corrupted files
corrupted_files = []

# Traverse through census block folders
for census_block in os.listdir(root_dir):
    census_block_path = os.path.join(root_dir, census_block)
    
    if os.path.isdir(census_block_path):  # Ensure it's a directory
        # Traverse through location subfolders
        for location in os.listdir(census_block_path):
            location_path = os.path.join(census_block_path, location)

            if os.path.isdir(location_path):  # Ensure it's a directory
                # Check PNG files inside
                for file in os.listdir(location_path):
                    if file.endswith(".png"):
                        file_path = os.path.join(location_path, file)
                        if os.path.getsize(file_path) <= 1024:  # File size ≤ 1KB
                            # Store path relative to Census block folder
                            relative_path = os.path.relpath(file_path, root_dir)
                            corrupted_files.append(relative_path)

# Print the corrupted files
if corrupted_files:
    print("Corrupted PNG files (≤ 1KB):")
    for file in corrupted_files:
        print(file)
else:
    print("No corrupted PNG files found.")

# Retrieve pair of images which are according to the folder structure.

In [ ]:
import os
import pandas as pd
from datetime import datetime

# Paths
base_dir = 'Streetview_data'
input_excel = 'Streetview Image Label.xlsx'
output_excel = 'revised_image_labels.xlsx'

def find_matching_folder(census_block_id, lat, lon, heading):
    """
    Find the best-matching folder inside the census block directory by comparing lat/lon up to the second-last digit.
    """
    base_folder = os.path.join(base_dir, str(census_block_id))
    if not os.path.exists(base_folder):
        return None  # If the census block folder doesn't exist, return None

    # Convert lat/lon to strings once
    lat_str, lon_str = str(lat), str(lon)

    # Precompute length for slicing
    lat_len, lon_len = len(lat_str) - 1, len(lon_str) - 1

    matching_folders = []

    with os.scandir(base_folder) as entries:
        directories = [entry for entry in entries if entry.is_dir()] # Skip files, only check directories
        for entry in directories:
            parts = entry.name.split("_")
            if len(parts) != 3: # escape invalid folders
                continue 
            try:
                folder_lat_str, folder_lon_str, folder_heading = str(parts[0]), str(parts[1]), int(parts[2])
                
                # Check up to second-last digit     
                if (folder_lat_str[:lat_len] == lat_str[:lat_len] and 
                    folder_lon_str[:lon_len] == lon_str[:lon_len] and 
                    folder_heading == heading):
                    matching_folders.append(entry.name)

                    if len(matching_folders) > 1:
                        print(entry.name)
                        raise Exception("More than two matching folders found.")

            except ValueError:
                continue  # Skip if conversion fails

    return matching_folders[0]  # return first folder


# Load the Excel file
df = pd.read_excel(input_excel, dtype={"Latitude": str, "Longitude": str})

# Convert Start and End columns to datetime with day first
df['Start'] = pd.to_datetime(df['Start'], dayfirst=True, errors='coerce').dt.date
df['End'] = pd.to_datetime(df['End'], dayfirst=True, errors='coerce').dt.date

# Prepare a list to collect valid records
valid_records = []

# Iterate through each record
for index, row in df.iterrows():
    census_block_id = row['ID']
    latitude = row['Latitude']
    longitude = row['Longitude']
    heading = row['Heading']
    start_date = datetime.strptime(str(row['Start']), '%Y-%m-%d').date()
    end_date = datetime.strptime(str(row['End']), '%Y-%m-%d').date()
    change = row['Change']

    # Construct folder path
    # folder_path = os.path.join(base_dir, str(census_block_id), f"{latitude}_{longitude}_{heading}")
    # Find the correct folder even if lat/lon precision is different
    folder = find_matching_folder(census_block_id, latitude, longitude, heading)
    if folder is None:
        raise FileNotFoundError(f"No matching folder found for {latitude}, {longitude}, {heading} in Census Block {census_block_id}")
    folder_path = os.path.join(base_dir, str(census_block_id), folder)
    # print(f"Folder: {census_block_id}/{folder} Start: {start_date} End: {end_date} Change: {change}")
    # Check if the folder exists
    if os.path.exists(folder_path):
        # List all images in the folder
        images = [img for img in os.listdir(folder_path) if img.endswith('.png')]

        # Extract dates from image filenames
        image_dates = {datetime.strptime(img.split('.')[0], '%b %Y').date(): img for img in images}

        # Check if both start and end images are present
        if start_date in image_dates and end_date in image_dates:
            lat, lng, heading = folder.split("_")
            row['Latitude'] = lat
            row['Longitude'] = lng
            valid_records.append(row)
        else:
            print(f"Folder: {census_block_id}/{folder} Start: {start_date} End: {end_date} Change: {change}")

# Save the valid records to a new Excel file
valid_df = pd.DataFrame(valid_records)

# Ensure Start and End columns are datetime objects without time
valid_df['Start'] = pd.to_datetime(valid_df['Start'], dayfirst=True).dt.date
valid_df['End'] = pd.to_datetime(valid_df['End'], dayfirst=True).dt.date

# Write to Excel with date format
with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
    valid_df.to_excel(writer, index=False)
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']

    # Apply date format without time
    date_format = workbook.add_format({'num_format': 'mm/dd/yyyy'})
    start_col_idx = valid_df.columns.get_loc('Start')
    end_col_idx = valid_df.columns.get_loc('End')

    # Apply format to columns
    worksheet.set_column(start_col_idx, start_col_idx, 15, date_format)
    worksheet.set_column(end_col_idx, end_col_idx, 15, date_format)

print(f"Validation complete. Saved {len(valid_records)} valid records to {output_excel}.")


# Make single sheet of population and area of years 2013 - 2019.

In [ ]:
# census data filename follows this structure ACSDT5Y2013.B01003-Data
# for all census data, columns: GEO_ID -> Census block id and B01003_001E -> Population
# for census block group area, filename -> mecklenburg_county_census_block_group_area.xlsx
# columns: GEOID, area_sq_km, area_sq_km, area_sq_m, area_statute_miles, area_us_survey_miles 

import pandas as pd

# Define file paths
file_list = [f"Total_Population_Census_Block/ACSDT5Y{year}.B01003-Data.csv" for year in range(2013, 2020)]
area_file = "mecklenburg_county_census_block_group_area.xlsx"

# Read census block area data
area_df = pd.read_excel(area_file, dtype={'GEOID': str})

# Initialize an empty DataFrame for population data
population_df = pd.DataFrame()

# Read and merge all population data files
for year, file in zip(range(2013, 2020), file_list):
    df = pd.read_csv(file, dtype={'GEO_ID': str})
    df['GEO_ID'] = df['GEO_ID'].str.replace('1500000US', '', regex=False)
    df = df[['GEO_ID', 'B01003_001E']].rename(columns={'GEO_ID': 'ID', 'B01003_001E': f'Pop_{year}'})
    
    if population_df.empty:
        population_df = df
    else:
        population_df = population_df.merge(df, on='ID', how='outer')

# Merge with area data
merged_df = population_df.merge(area_df, left_on='ID', right_on='GEOID', how='left').drop(columns=['GEOID', 'NAMELSAD', 'INTPTLAT', 'INTPTLON', 'Shape_Length'])

# Convert all columns to numeric
merged_df = merged_df.apply(pd.to_numeric, errors='coerce')

# Save to an Excel file
output_file = "merged_census_data.xlsx"
merged_df.to_excel(output_file, index=False, float_format="%.10f")

print(f"Merged data saved to {output_file}")


# Make single sheet of population and area of years 2020 - 2023.

In [ ]:
import pandas as pd

# Define file paths
file_list = [f"Total_Population_Census_Block/ACSDT5Y{year}.B01003-Data.csv" for year in range(2020, 2024)]
area_file = "mecklenburg_county_census_block_group_area_2020.xlsx"

# Read census block area data
area_df = pd.read_excel(area_file, dtype={'GEOID': str})

# Initialize an empty DataFrame for population data
population_df = pd.DataFrame()

# Read and merge all population data files
for year, file in zip(range(2020, 2024), file_list):
    df = pd.read_csv(file, dtype={'GEO_ID': str})
    df['GEO_ID'] = df['GEO_ID'].str.replace('1500000US', '', regex=False)
    df = df[['GEO_ID', 'B01003_001E']].rename(columns={'GEO_ID': 'ID', 'B01003_001E': f'Pop_{year}'})
    
    if population_df.empty:
        population_df = df
    else:
        population_df = population_df.merge(df, on='ID', how='outer')

# Merge with area data
merged_df = population_df.merge(area_df, left_on='ID', right_on='GEOID', how='left').drop(columns=['GEOID', 'NAMELSAD', 'INTPTLAT', 'INTPTLON', 'Shape_Length'])

# Convert all columns to numeric
merged_df = merged_df.apply(pd.to_numeric, errors='coerce')

# Save to an Excel file
output_file = "merged_census_data_2020.xlsx"
merged_df.to_excel(output_file, index=False, float_format="%.10f")

print(f"Merged data saved to {output_file}")


# Filter revised image labels in between 2013 and 2019 including the edge years.

In [ ]:
import pandas as pd

# Load the Excel file with correct data types
file_path = "revised_image_labels.xlsx"
df = pd.read_excel(file_path, dtype={"Latitude": str, "Longitude": str})

# Convert 'Start' and 'End' columns to datetime format with day-first parsing
df["Start"] = pd.to_datetime(df["Start"], dayfirst=True, errors="coerce")
df["End"] = pd.to_datetime(df["End"], dayfirst=True, errors="coerce")

# Check the distribution of categories
category_counts = df["Change"].value_counts()
print("Original Category Distribution:\n", category_counts)

# Filter out rows where 'Start' or 'End' year is outside the range 2013-2019
filtered_df = df[(df["Start"].dt.year.between(2013, 2019)) & (df["End"].dt.year.between(2013, 2019))]

# Count occurrences of each 'Change' category
change_counts = filtered_df["Change"].value_counts()

# Save the filtered data back to a new Excel file with proper date formatting
output_path = "image_labels_2013_to_2019.xlsx"
with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    filtered_df.to_excel(writer, index=False)
    workbook = writer.book
    worksheet = writer.sheets["Sheet1"]

    # Apply date format to ensure MM/DD/YYYY formatting in Excel
    date_format = workbook.add_format({"num_format": "mm/dd/yyyy"})
    start_col_idx = filtered_df.columns.get_loc("Start")
    end_col_idx = filtered_df.columns.get_loc("End")

    worksheet.set_column(start_col_idx, start_col_idx, 15, date_format)
    worksheet.set_column(end_col_idx, end_col_idx, 15, date_format)

# Print results
print(f"Filtered data saved to '{output_path}'\n")
print("Change category counts after filtering:")
print(change_counts)


# Link 2020 census block group to 2013 census block group

In [ ]:
import geopandas as gpd
import pandas as pd
import os

bg_2013 = gpd.read_file("mecklengburg_county_census_block_group_2013/mecklengburg_county_census_block_group_2013.shp")
bg_2020 = gpd.read_file("mecklengburg_county_census_block_group_2020/mecklengburg_county_census_block_group_2020.shp")

bg_2013 = bg_2013[['GEOID', 'geometry']].rename(columns={'GEOID': 'GEOID_2013'})
bg_2020 = bg_2020[['GEOID', 'geometry']].rename(columns={'GEOID': 'GEOID_2020'})

intersections = gpd.overlay(bg_2020, bg_2013, how='intersection')

intersections['intersect_area'] = intersections.area

max_overlap = (
    intersections
    .sort_values(by='intersect_area', ascending=False)
    .drop_duplicates(subset='GEOID_2020')
    .loc[:, ['GEOID_2020', 'GEOID_2013', 'intersect_area']]
)
# Convert all columns to numeric
max_overlap = max_overlap.apply(pd.to_numeric, errors='coerce')
max_overlap.to_excel("bg_2020_to_2013_mapping.xlsx", index=False, float_format="%.14f")

# bg_2020_mapped = bg_2020.merge(max_overlap[['GEOID_2020', 'GEOID_2013']], on='GEOID_2020', how='left')
# os.makedirs("mapped_shapefile", exist_ok=True)
# bg_2020_mapped.to_file("mapped_shapefile/bg_2020_with_2013_mapping.shp", driver='ESRI Shapefile')


# Change the blocks for prepared census data.

In [ ]:
# mapping between the census block groups was created above.
# Now, let's actually change the blocks for prepared census data.

import pandas as pd

# Load the data
prev_census_data_path = "merged_census_data_2020.xlsx"
mapping_census_blocks_path = "bg_2020_to_2013_mapping.xlsx"

census_df = pd.read_excel(prev_census_data_path)
block_mapping_df = pd.read_excel(mapping_census_blocks_path)

# Merge the census data with the mapping data
merged_df = census_df.merge(block_mapping_df, left_on="ID", right_on="GEOID_2020", how="left")

# Locate the row with ID 371190001041
target_row = merged_df[merged_df["ID"] == 371190001041]

# Create two new rows with split values
row_1 = target_row.copy()
row_2 = target_row.copy()

# Update IDs
row_1["GEOID_2013"] = 371190001003
row_2["GEOID_2013"] = 371190001004

# Define columns to split
pop_columns = [f"Pop_{year}" for year in range(2020, 2024)]
popden_columns = [f"PopDen_{year}" for year in range(2020, 2024)]

# Apply the percentage splits
row_1[pop_columns + popden_columns] *= 0.32
row_2[pop_columns + popden_columns] *= 0.68

# Remove the original row
merged_df = merged_df[merged_df["ID"] != 371190001041]

# Append the two new rows
final_df = pd.concat([merged_df, row_1, row_2], ignore_index=True)
final_df = final_df.drop(columns=["ID", "STATEFP", "COUNTYFP", "TRACTCE", "BLKGRPCE", "MTFCC", "FUNCSTAT", "ALAND","AWATER", "GEOID_2020", "intersect_area"]).rename(columns={'GEOID_2013': 'ID'})

# Group by 'ID' and sum all other columns
aggregated_df = final_df.groupby("ID", as_index=False).sum()

# # Save to an Excel file
output_file = "census_density_calculate_2020_with_2013_blocks.xlsx"
aggregated_df.to_excel(output_file, index=False, float_format="%.10f")
print(f"Merged data saved to {aggregated_df}")



# Merge these census density calculated 2020 with census density calculated to make data from 2013 to 2023

In [ ]:
import pandas as pd

census_density_cal_2013 = "census_density_calculated.xlsx"
census_density_cal_2020 = "census_density_calculate_2020_with_2013_blocks.xlsx"

density_cal_2013_df = pd.read_excel(census_density_cal_2013)
density_cal_2020_df = pd.read_excel(census_density_cal_2020)

# Merge the census data with the mapping data
merged_df = density_cal_2013_df.merge(density_cal_2020_df, left_on="ID", right_on="ID", how="left")

merged_df = merged_df.drop(columns=['area_sq_m_x', 'area_statute_miles', 'area_us_survey_miles', 'Shape_Area_y', 'area_sq_m_y'])

new_order = ['ID', 'Pop_2013', 'Pop_2014', 'Pop_2015', 'Pop_2016', 'Pop_2017',
       'Pop_2018', 'Pop_2019', 'Pop_2020', 'Pop_2021', 'Pop_2022', 'Shape_Area_x', 'area_sq_km', 'PopDen_2013',
       'PopDen_2014', 'PopDen_2015', 'PopDen_2016', 'PopDen_2017',
       'PopDen_2018', 'PopDen_2019','PopDen_2020', 'PopDen_2021',
       'PopDen_2022', 'PopDen_2023']
merged_df = merged_df.reindex(columns=new_order)
# Save to an Excel file
output_file = "census_density_calculated_2013_2023.xlsx"
merged_df.to_excel(output_file, index=False, float_format="%.10f")
print(f"Merged data saved to {merged_df}")

# Filter revised image labels in between 2013 and 2023 including the edge years.

In [ ]:
import pandas as pd

# Load the Excel file with correct data types
file_path = "revised_image_labels.xlsx"
df = pd.read_excel(file_path, dtype={"Latitude": str, "Longitude": str})

# Convert 'Start' and 'End' columns to datetime format with day-first parsing
df["Start"] = pd.to_datetime(df["Start"], dayfirst=True, errors="coerce")
df["End"] = pd.to_datetime(df["End"], dayfirst=True, errors="coerce")

# Check the distribution of categories
category_counts = df["Change"].value_counts()
print("Original Category Distribution:\n", category_counts)

# Filter out rows where 'Start' or 'End' year is outside the range 2013-2023
filtered_df = df[(df["Start"].dt.year.between(2013, 2023)) & (df["End"].dt.year.between(2013, 2023))]

# Count occurrences of each 'Change' category
change_counts = filtered_df["Change"].value_counts()

# Save the filtered data back to a new Excel file with proper date formatting
output_path = "image_labels_2013_to_2023.xlsx"
with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    filtered_df.to_excel(writer, index=False)
    workbook = writer.book
    worksheet = writer.sheets["Sheet1"]

    # Apply date format to ensure MM/DD/YYYY formatting in Excel
    date_format = workbook.add_format({"num_format": "mm/dd/yyyy"})
    start_col_idx = filtered_df.columns.get_loc("Start")
    end_col_idx = filtered_df.columns.get_loc("End")

    worksheet.set_column(start_col_idx, start_col_idx, 15, date_format)
    worksheet.set_column(end_col_idx, end_col_idx, 15, date_format)

# Print results
print(f"Filtered data saved to '{output_path}'\n")
print("Change category counts after filtering:")
print(change_counts)


# Integrate image labels with population density data.

In [ ]:
import pandas as pd

# File paths
census_file = "image_labels_2013_to_2023.xlsx"
pop_density_file = "census_density_calculated_2013_2023.xlsx"

# Load the census data
df_census = pd.read_excel(census_file, dtype={"Latitude": str, "Longitude": str})

# Load the population density data
df_pop = pd.read_excel(pop_density_file, dtype={"ID": str})  # Ensure ID is string to match

# Convert 'Start' and 'End' columns to datetime format
df_census["Start"] = pd.to_datetime(df_census["Start"], dayfirst=True, errors="coerce")
df_census["End"] = pd.to_datetime(df_census["End"], dayfirst=True, errors="coerce")

# Extract years from 'Start' and 'End' columns
df_census["Start_Year"] = df_census["Start"].dt.year
df_census["End_Year"] = df_census["End"].dt.year

# Function to get population density difference
def get_pop_density_diff(row):
    block_id = str(row["ID"])  # Ensure ID is string to match both dataframes
    start_year = row["Start_Year"]
    end_year = row["End_Year"]

    # Fetch the population density values for the start and end years
    pop_start = df_pop.loc[df_pop["ID"] == block_id, f"PopDen_{start_year}"]
    pop_end = df_pop.loc[df_pop["ID"] == block_id, f"PopDen_{end_year}"]

    # Check if values exist before computing the difference
    if not pop_start.empty and not pop_end.empty:
        popden_diff = pop_end.values[0] - pop_start.values[0]
        if pop_end.values[0] == 0 and pop_start.values[0] == 0:
            popden_diff_percentage = 0
        elif pop_start.values[0] == 0:
            popden_diff_percentage = 100
        else:
            popden_diff_percentage = (popden_diff * 100) / pop_start.values[0]
        return popden_diff, popden_diff_percentage
    return None, None

# Apply the function to compute population density difference
# df_census["PopDen_Diff"] = df_census.apply(get_pop_density_diff, axis=1)
df_census[["PopDen_Diff", "PopDen_Diff_Percentage"]] = df_census.apply(
    lambda row: pd.Series(get_pop_density_diff(row)), axis=1
)
df_census.drop(columns=['Start_Year', 'End_Year'], inplace=True)
# Save the updated census data with the new column
output_file = "image_labels_2013_to_2023_with_pop_den.xlsx"
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    df_census.to_excel(writer, index=False)
    workbook = writer.book
    worksheet = writer.sheets["Sheet1"]

    # Apply date format to ensure MM/DD/YYYY formatting in Excel
    date_format = workbook.add_format({"num_format": "mm/dd/yyyy"})
    start_col_idx = df_census.columns.get_loc("Start")
    end_col_idx = df_census.columns.get_loc("End")

    worksheet.set_column(start_col_idx, start_col_idx, 15, date_format)
    worksheet.set_column(end_col_idx, end_col_idx, 15, date_format)

# Print message
print(f"Updated census data saved to '{output_file}'")



# Sub sample the dataset.

In [ ]:
import pandas as pd

# Load the dataset
excel_file = "image_labels_2013_to_2023_with_pop_den.xlsx"
df = pd.read_excel(excel_file, dtype={"Latitude": str, "Longitude": str})

# Convert Start and End to datetime
df['Start'] = pd.to_datetime(df['Start'], dayfirst=True, errors='coerce').dt.date
df['End'] = pd.to_datetime(df['End'], dayfirst=True, errors='coerce').dt.date

# Exclude re-greening
df = df[df["Change"] != "re-greening"]

category_counts = df["Change"].value_counts()
print("Original Category Distribution excluding re-greening:\n", category_counts)

# Separate "No Change"
df_no_change = df[df["Change"] == "No Change"].reset_index(drop=True)

# Sample every 10th row from "No Change"
df_no_change_sampled = df_no_change.iloc[::10].head(5580)

# Keep other categories as-is
df_others = df[df["Change"] != "No Change"]

# Combine datasets
df_final_sampled = pd.concat([df_no_change_sampled, df_others], ignore_index=True)

# Duplicate "re-capital" rows two times
df_re_capital = df_final_sampled[df_final_sampled["Change"] == "re-capital"]
df_re_capital_duplicated = pd.concat([df_re_capital] * 2, ignore_index=True)

# Remove original "re-capital" rows and add duplicated rows
df_final_sampled = df_final_sampled[df_final_sampled["Change"] != "re-capital"]
df_final_sampled = pd.concat([df_final_sampled, df_re_capital_duplicated], ignore_index=True)

# Sort the dataframe
df_final_sampled = df_final_sampled.sort_values(
    by=["ID", "Latitude", "Longitude", "Heading", "Start"],
    ascending=[True, True, True, True, True]
).reset_index(drop=True)

# Save to Excel with formatting
output_excel = "every_10_nochange_recapital_doubled_2013_2023_pop_den_labels.xlsx"
with pd.ExcelWriter(output_excel, engine="xlsxwriter") as writer:
    df_final_sampled.to_excel(writer, index=False)
    workbook = writer.book
    worksheet = writer.sheets["Sheet1"]

    # Date formatting
    date_format = workbook.add_format({"num_format": "mm/dd/yyyy"})
    worksheet.set_column(df_final_sampled.columns.get_loc("Start"), df_final_sampled.columns.get_loc("Start"), 15, date_format)
    worksheet.set_column(df_final_sampled.columns.get_loc("End"), df_final_sampled.columns.get_loc("End"), 15, date_format)

# Final distribution check
print(df_final_sampled["Change"].value_counts())
